# LAB 07 - Random Forest for Regression

In this lab we will be extending the previous lab about Decision trees and build a Regression model using Random Forest.

For simplicity, we will be using the same dataset as the previous lab (you can find it in ECLASS).

**IMPORTANT:** For this lab, if you haven't finished your code from last week's lab on Decision trees, you will have the option to use the sklearn implementation for a regression tree. However, this doesn't mean that you should skip the previous lab. This is just so that you don't get behind with the content and you don't spend all your time today working on the previous lab. 

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
from sklearn.model_selection import train_test_split

As mentioned before, use the Boston Housing data and prepare your train/val/test split as usual.

## Exercise 1 -- Bootstrap

Also known as [bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating), this technique consists of making several samples with replacement of the original data, using each of the samples to train an estimator, and then aggregating the predictions using the average (this is also a type of model ensemble).

In [2]:
def bootstrap(X, num_bags=10):
    """
    Given a dataset and a number of bags,
    sample the dataset with replacement.
    
    This function does not return a copy
    of the datapoints, but a list of indices
    with compatible dimensionality
    
    Parameters
    ----------
    X : ndarray
        A dataset
    num_bags : int, default 10
        The number of bags to create
    
    Returns
    -------
    list of ndarray
        The list contains `num_bags` integer one-dimensional ndarrays.
        Each of these contains the indices corresponding to the 
        sampled datapoints in `X`
    
    Notes
    -----
    * The number of datapoints in each bach will
      match the number of datapoints in the given
      dataset.
    * The
    """
    rng = np.random.default_rng(0) # you can change the seed, or use 0 to replicate my results
    # Your code here

    # Número de datapoints
    n_of_datapoints = X.shape[0]
    # Array com os índices dos dados
    indices = np.array(range(n_of_datapoints))
    # Lista com os bags criados
    bags = []

    # Para cada bag...
    for _ in range(num_bags):
        # Sorteia as amostras aleatoriamente
        samples = rng.choice(indices, size = n_of_datapoints, replace = True)
        # Adiciona-as na lista
        bags.append(samples)

    return bags

In [3]:
rng = np.random.default_rng(0)
X_small = rng.random(size=(100,2))
bags = bootstrap(X_small)
bags[0]

array([85, 63, 51, 26, 30,  4,  7,  1, 17, 81, 64, 91, 50, 60, 97, 72, 63,
       54, 55, 93, 27, 81, 67,  0, 39, 85, 55,  3, 76, 72, 84, 17,  8, 86,
        2, 54,  8, 29, 48, 42, 40,  2,  0, 12,  0, 67, 52, 64, 25, 61, 76,
       38, 46, 99, 80, 98, 37, 68, 95, 65, 84, 68, 70, 38, 87, 13, 57, 72,
       84, 52, 37, 31, 42, 48, 71, 88,  7, 93, 53, 35, 67, 57, 25, 32, 71,
       59, 50, 33, 76, 39, 32, 89, 26, 22, 71, 62,  4,  8, 37, 83])

## Exercise 2 -- Aggregation

The second part of bagging.

In [4]:
def aggregate_regression(preds):
    """
    Aggregate predictions by several estimators
    
    Parameters
    ----------
    preds : list of ndarray
        Predictions from multiple estimators.
        All ndarrays in this list should have the same
        dimensionality.
        
    Return
    ------
    ndarray
        The mean of the predictions
    """
    # Your code here

    # Criando um único array com todas as predições
    preds = np.array(preds)
    # Calculando as previsões médias
    mean_preds = np.mean(preds, axis = 0)

    return mean_preds

## Exercise 3 -- Random Forest for regression

Using the functions you implemented above, it is now time to put all of them together to train several decision trees and then ensemble them to output a single prediction. For the random forest, however, we need to select a subset of features at each split on the decision tree. 

For this part, you can use the sklearn implementation of Random forest for regression as your estimator for each set of features and bags. See below an example of how to do this, and always remember to check the necessary documentation when using an external function.

Some parameters you will have to set are: 
* num_features: number of features per estimator
* min_samples: min number of samples per leaf node
* max_depth: maximum depth of the decision tree (each estimator)
* num_estimators: number of decision trees you will create using each bag and random set of features

In [5]:
# example of sklearn Decision tree
# estimator = DecisionTreeRegressor(max_depth=self.max_depth)
# estimator.fit(X, y)
# estimator.predict(X)

In [6]:
## your code goes here:

# Carregando e separando os dados
data = pd.read_csv("BostonHousing.txt")

y = data["medv"].to_numpy()
X = data.drop("medv", axis = 1).to_numpy()

X_train_and_val, X_test, y_train_and_val, y_test = train_test_split(X, y, test_size = 0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train_and_val, y_train_and_val, test_size = 1/9)

In [7]:
# Função para calcular o erro de criterion de uma região
def regression_criterion(region):
    # Se a região for vazia, o erro é infinito
    if region.size == 0:
        return float("inf")
    
    # Calculando a média da região
    y_hat = np.mean(region)
    # Calculando o erro da região
    region_error = np.sum((region - y_hat)**2)

    return region_error


# Função para dividir uma região
def split_region(region, feature_index, tau):
    # Array de booleanos para filtrar os registros
    split = region[:, feature_index] < tau
    # Array com os índices dos registros nessa região
    indices = np.array(range(region.shape[0]))

    # Encontrando as partições
    left_partition = indices[split]
    right_partition = indices[~split]

    return left_partition, right_partition


# Função para obter a melhor divisão de uma região
def get_split(X, y, available_features):
    # Inicializando as melhores configurações de divisão
    best_feature = 0
    best_tau = 0
    best_criterion = float("inf")

    # Para cada feature...
    for feature in available_features:
        # Cria a lista dos taus a serem testados, sendo os valores desses dados para essa feature
        # adicionado com um valor maior que todos eles
        taus = np.unique(X[:, feature])
        taus = np.append(taus, np.max(taus) + 1)

        # Para cada tau...
        for tau in taus:
            # Divide a região
            left, right = split_region(X, feature, tau)
            # Calcula o criterion somando os criterions das duas regiões criadas
            criterion = regression_criterion(y[left]) + regression_criterion(y[right])

            # Se esse criterion for menor que o melhor até agora...
            if criterion < best_criterion:
                # Atualiza as melhores configurações para essa
                best_feature = feature
                best_tau = tau
                best_criterion = criterion

    # Dividindo a região com as melhores configurações
    left_region, right_region = split_region(X, best_feature, best_tau)

    # Criando o dicionário com as informações solicitadas
    decision = {"feature_index": best_feature, "tau": best_tau, "left_region": left_region, "right_region": right_region}

    return decision


# Função para crescer uma árvore recursivamente
def recursive_growth(node, min_samples, max_depth, current_depth, X, y, available_features):
    # Para cada região...
    for region in ["left_region", "right_region"]:
        # Pega os índices dos registros dessa região
        region_indices = node[region]
        # Se algum dos critérios de parada for atendido...
        if region_indices.size <= min_samples or current_depth == max_depth or regression_criterion(y[region_indices]) == 0:
            # Esse nó é uma folha e passa a armazenar apenas seu valor
            new_node = {"value": np.mean(y[region_indices])}
        # Se não...
        else:
            # Faz uma nova divisão
            new_node = get_split(X[region_indices], y[region_indices], available_features)
            # E começa a crescer recursivamente a partir desse nó
            recursive_growth(new_node, min_samples, max_depth, current_depth + 1, X[region_indices], y[region_indices], available_features)

        # Se estamos falando da região da esquerda...
        if region == "left_region":
            # Adiciona esse novo nó como o filho da esquerda do nó atual
            node['left'] = new_node
        # Se não...
        else:
            # Adiciona-o como o filho da direita
            node['right'] = new_node


# Função para fazer a predição de um dado
def predict_sample(node, sample):
    # Enquanto não chegar a uma folha...
    while "value" not in node.keys():
        # Se o valor desse datapoint para a feature desse nó for menor que o limiar desse nó...
        if sample[node["feature_index"]] < node["tau"]:
            # Vai para o nó da esquerda
            node = node["left"]
        # Se não...
        else:
            # Vai para o da direita
            node = node["right"]

    # A previsão é o valor da folha alcançada
    pred = node["value"]

    return pred

# Função para fazer a predição de um conjunto de dados
def predict(node, X):
    # Número de dados
    n_of_datapoints = X.shape[0]

    # Array para armazenar as previsões
    y_pred = np.zeros(n_of_datapoints)

    # Para cada dado...
    for datapoint in range(n_of_datapoints):
        # Faz sua previsão e armazena-a no array
        pred = predict_sample(node, X[datapoint])
        y_pred[datapoint] = pred

    return y_pred

In [8]:
# Função para treinar uma floresta aleatória
def train_random_forest(X, y, n_of_features, min_samples, max_depth, n_of_estimators):
    # Criando os bags por meio do bootstrap
    bags = bootstrap(X, n_of_estimators)
    # Lista para armazenar as árvores
    random_forest = []

    # Para cada árvore...
    for i in range(n_of_estimators):
        # Sorteia as features que serão utilizadas
        features = np.random.choice(np.array(range(X.shape[1])), n_of_features, replace = False)
        # Filtra os dados apenas com os registros sorteados
        filtered_X = X[bags[i]]
        # Gera a árvore
        root = get_split(filtered_X, y[bags[i]], features)
        recursive_growth(root, min_samples, max_depth, 1, filtered_X, y[bags[i]], features)
        # Salva-a na lista
        random_forest.append(root)

    return random_forest


# Função para fazer a predição com base na floresta aleatória
def predict_random_forest(random_forest, X):
    # Matriz para armazenar as predições das árvores
    preds = []

    # Para cada árvore...
    for i in range(len(random_forest)):
        # Faz a predição dos dados
        pred = predict(random_forest[i], X)
        # Insere-a na matriz
        preds.append(pred)

    # Calcula a predição final
    final_pred = aggregate_regression(preds)

    return final_pred


# Função para calcular o RMSE
def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true)**2).mean())

In [9]:
random_forest = train_random_forest(X_train, y_train, round(np.sqrt(X.shape[1])), 20, 6, 10)
val_pred = predict_random_forest(random_forest, X_val)
rmse(y_val, val_pred)

4.570810054159512

In [10]:
# Testando para várias configurações
nums_features = [3, 4, 5]
mins_samples = [15, 20, 25]
max_depths = [4, 6, 8]
nums_estimators = [5, 10, 15]

best_configuration = [0, 0, 0, 0]
best_error = np.inf

for each_num_features in nums_features:
    for each_min_samples in mins_samples:
        for each_max_depth in max_depths:
            for each_num_estimators in nums_estimators:
                random_forest = train_random_forest(X_train, y_train, each_num_features, each_min_samples, each_max_depth, each_num_estimators)
                val_pred = predict_random_forest(random_forest, X_val)
                error = rmse(y_val, val_pred)
                if error < best_error:
                    best_configuration = [each_num_features, each_min_samples, each_max_depth, each_num_estimators]
                    best_error = error

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [11]:
print(f"Best error: {best_error}\n  N° of features: {best_configuration[0]}\n  Min samples: {best_configuration[1]}\n  Max depth: {best_configuration[2]}\n  N° of estimators: {best_configuration[3]}")

Best error: 3.3547049752519995
  N° of features: 5
  Min samples: 25
  Max depth: 8
  N° of estimators: 5
